# Prediction of Scores for Map Ranking

We'll proceed with using the module to predict all unknown scores in order to rank maps.

We have sufficient evidence of decent accuracy prediction of accuracies, but, the translation of these accuracies to map ranking will require research.

## Preparation

As usual, we'll load relevant libraries and ...
- Load our test data
- Load our model

In [1]:
import sys

sys.path.append("../opal/data_ppy_sh_to_csv/")
sys.path.append("../")

from opal.score.collaborative_filtering.lit_neu_mf_net import LitNeuMFNet
from opal.score.dataset.datamodule import ScoreDataModule

dm = ScoreDataModule(
    ds_yyyy_mm="2022_12", batch_size=256, m_min_support=50, u_min_support=50,
    score_bounds=(7.5e5, 1e6),
)
dm.setup()

Download Files: 
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000.tar.bz2
Derived Files: 
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000/___.sql (All SQL Files are extracted)
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000/csv/osu_user_stats_mania.csv
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000/csv/osu_scores_mania_high.csv
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000/csv/osu_beatmap_difficulty.csv
	- E:/Repositories/PycharmProjects/opal/models/../data/2022_12_01_performance_mania_top_1000/csv/osu_beatmaps.csv
Proceeding to Download ...
File exists, and overwrite is false, skipping


E:\Repositories\PycharmProjects\opal\models\..\opal\score\dataset\datamodule.py:74: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_map = pd.read_csv(csv_map)


In [2]:
net = LitNeuMFNet.load_from_checkpoint(
    "tiny/checkpoints/epoch=2-step=6144.ckpt",
    # "version_256_256_32_acc/checkpoints/epoch=6-step=14336.ckpt",
    uid_no=dm.n_uid,
    mid_no=dm.n_mid,

    mf_emb_dim=64,
    mlp_emb_dim=64,
    mlp_chn_out=8,
    # mf_emb_dim=256,
    # mlp_emb_dim=256,
    # mlp_chn_out=32,
    scaler=dm.scaler_accuracy,
    lr=0.005
)
# Set to evaluate mode (no gradient update)
net.eval()

LitNeuMFNet(
  (model): NeuMFNet(
    (u_mf_emb): Embedding(4007, 64)
    (m_mf_emb): Embedding(6189, 64)
    (mf_net): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
    )
    (u_mlp_emb): Embedding(4007, 64)
    (m_mlp_emb): Embedding(6189, 64)
    (mlp_net): Sequential(
      (0): Linear(in_features=128, out_features=512, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=256, out_features=128, bias=True)
      (7): ReLU()
      (8): Dropout(p=0.1, inplace=False)
      (9): Linear(in_features=128, out_features=64, bias=True)
      (10): ReLU()
      (11): Dropout(p=0.1, inplace=False)
      (12): Linear(in_feat

## Yielding unknown scores


In [94]:
df = dm.df

In [95]:
df.columns

Index(['difficultyrating', 'diff_overall', 'diff_size', 'version',
       'beatmap_id', 'filename', 'user_id', 'year', 'score', 'accuracy',
       'speed', 'uid', 'mid', 'uid_le', 'mid_le'],
      dtype='object')

In [96]:
df = df[(df['speed'] == 0) & (df['year'] == '2019')].astype({
    'beatmap_id': int,
    'user_id': int
}).set_index(['beatmap_id', 'user_id'])

In [97]:
df_g = df.groupby(['beatmap_id', 'user_id']).agg({'accuracy': 'mean'})

In [98]:
df_p = df_g.reset_index().astype({
    'beatmap_id': str,
    'user_id': str
}
).assign(
    beatmap_id=lambda x: x.beatmap_id + "/0",
    user_id=lambda x: x.user_id + "/2019"
).pivot(index='beatmap_id', columns='user_id', values='accuracy')

In [99]:
df_m = df_p.reset_index().melt(id_vars='beatmap_id', value_name='accuracy')
# df_m = df_m[df_m['accuracy'].isna()]

In [100]:
df_m['mid'] = dm.mid_le.transform(df_m[['beatmap_id']].values)
df_m['uid'] = dm.uid_le.transform(df_m[['user_id']].values)

E:\ProgramFiles\Anaconda\envs\opal\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
E:\ProgramFiles\Anaconda\envs\opal\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [101]:
import torch

from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl

ar_uid = torch.tensor(df_m[['uid']].values)
ar_mid = torch.tensor(df_m[['mid']].values)
ar_dummy = torch.ones_like(ar_uid, dtype=float)
ds = TensorDataset(ar_uid, ar_mid, ar_dummy)
dl = DataLoader(ds, batch_size=128)

trainer = pl.Trainer(
    accelerator='cpu',
    # limit_predict_batches=16,
)

pred = trainer.predict(net, dataloaders=dl)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: E:\Repositories\PycharmProjects\opal\models\lightning_logs
E:\ProgramFiles\Anaconda\envs\opal\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [102]:
import numpy as np

df_m['pred'] = np.concatenate([p for p, _ in pred])[:, 0]

In [119]:
df_pp = df_m.pivot(index='beatmap_id', columns='user_id', values=['pred']).mean(axis=1).rename(
    'accuracy'
).reset_index().assign(beatmap_id=lambda x: x.beatmap_id.str[:-2].astype(int))

In [145]:
df_pred = df.merge(df_pp, on='beatmap_id')[
    ['difficultyrating', 'filename', 'accuracy_y']
].drop_duplicates().assign(
    cf_rank=lambda x: x['accuracy_y'].rank().astype(int),
    sr_rank=lambda x: x['difficultyrating'].rank(ascending=False).astype(int),
    filename=lambda x: x['filename'].str[:-4]
).drop('accuracy_y', axis=1).rename({
    'difficultyrating': 'sr',
    'filename': 'beatmap'
}, axis=1).assign(
    change=lambda x: x.sr_rank - x.cf_rank
)[['cf_rank', 'sr_rank', 'change', 'sr', 'beatmap']].sort_values('cf_rank')

In [147]:
df_pred.to_clipboard(index=False)